In [ ]:
# default_exp core.baseline

## Baseline Analysis

> We are using the adjudidicated cases from 2018 to differentiate the cases by their Origin.  Our first step is to determine if the they have different status counts.  The conclusion of this analysis will driver futher analysis of years where there were significant policy changes made due to COVID.


In particular ...

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'



In [ ]:
#export
import pandas as pd
import altair as alt


In [ ]:
#export
ORIGIN_MAPPINGS = {
        'DMER - DRIVER MEDICAL EXAMINATION REPORT': 'DMER', 
        'OTHER':'Other', 
        'PHYSICIAN':'Physician',
        'PUBLIC/FAMILY': 'Public/Family', 
        'EVF - EXAM OF VISUAL FUNCTIONS': 'EVF',
        'ICBC TRIGGER':'ICBC Trigger',
        'ICBC':'ICBC',
        'OTHER HEALTH PROFESSIONAL':'Other Health', 
        'POLICE':'Police', 
        'REHAB':'Rehab',
        'DRIVER IMPROVEMENT PROGRAM':'Driver Improvement',
        'DRIVER APPEAL REQUEST':'Appeal',
        'SEAT BELT EXEMPTION APPLICATION':'Seat Belt',
        'SPECIALIST REPORT':'Specialist Report',
        'MLA ENQUIRY':'MLA Enquiry'    
}


In [ ]:
#export

def get_fte_days_2018():
# We created three lists, containing names, ages, and locations, holding our ordered data
# We then created a Python zip() object, which contained the zips of names, ages, and locations. We then applied the list() function to turn this zip object into a list of tuples
# We then passed this zipped object into our DataFrame() class, along with a list of our column names to create our dataframe.

    months = ['Jan-2018', 'Feb-2018', 'Mar-2018', 'Apr-2018','May-2018', 'Jun-2018', 'Jul-2018', 'Aug-2018','Sep-2018', 'Oct-2018', 'Nov-2018', 'Dec-2018']
    ftes = [376
    ,311
    ,361
    ,332
    ,394
    ,360
    ,335
    ,366
    ,323
    ,380
    ,335
    ,281]


    zipped = list(zip(months, ftes))
    ftedays_df = pd.DataFrame(zipped, columns=['Month-Year', 'FTE (Days)'])
    ftedays_df

    d = ftedays_df.copy()
    d.columns = ['Month', 'FTE (Days)']
    
    return d


In [ ]:
#export
def derive_statuscount_percase(df_in, ftedays_df):
    FTE_DAYS_YEAR = ftedays_df['FTE (Days)'].sum()
    print('FTE_DAYS_YEAR', FTE_DAYS_YEAR)
    aggregation = {
        'Case Count': ('STATUS_COUNT','size'),
        'Total Status Change Count': ('STATUS_COUNT','sum')
    }    

    # df = df_in.groupby(['ORIGIN_DSC']) \
    # .agg(** aggregation).reset_index()

    # aggregation = {
    #     'Case Count': ('DRIVERS_LICENSE_NO','size'),
    #     'Status Count': ('STATUS_COUNT','sum'),
    #     }

    df = df_in.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                                pd.Grouper(key='ORIGIN_DSC'),                                   
                                ]).agg(** aggregation)

    df = pd.DataFrame(df).reset_index()

    df['Opened Month'] = df.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)
    
    #case_summary_data['Total Cases in Group'] = case_summary_data.groupby(['Origin Report'])['Cases'].transform(lambda x: sum(x) )
    df['Total Cases In Month'] = df.groupby(['CASE_OPENED_DT'])['Case Count'].transform( lambda x: sum(x))
    df['Total Status Changes In Month'] = df.groupby(['CASE_OPENED_DT'])['Total Status Change Count'].transform( lambda x: sum(x))
    df['Monthly Status Changes/Case'] = df.apply(lambda x: x['Total Status Changes In Month']/x['Total Cases In Month']  , axis=1)


    df['Group Status Changes/Case'] = df.apply(lambda x: x['Total Status Change Count']/x['Case Count']  , axis=1)
    #Month-Year	FTE (Days)
    df = pd.merge(df, ftedays_df, how='left',  left_on='Opened Month', right_on='Month')
    #df = df[df['Is Adjudicated'] == 'Adjudicated']
    print('df shape ', df.shape)
    case_count = df['Case Count'].sum()
    status_change_count = df['Total Status Change Count'].sum()

    df['Group Case Count/FTE'] = df.apply(lambda x: x['Case Count']/x['FTE (Days)']  , axis=1)
    df['Group Status Change/FTE'] = df.apply(lambda x: x['Total Status Change Count']/x['FTE (Days)']  , axis=1)


    print(f"Adjudicated Case Count for 2018: {case_count:,}")
    print(f"Adjudicated Status Change Count for 2018: {status_change_count:,}" )
    print(f"Average Status Change Count/Case: {status_change_count/case_count : .2f}" )

    print(f"Adjudicated Cases/FTE Day: {case_count/FTE_DAYS_YEAR : .2f}") 
    print(f"Status Changes/FTE Day: { status_change_count/FTE_DAYS_YEAR: .2f}")
#    print(f"Monthly Team Capacity (Status Changes) { (status_change_count/FTE_DAYS_YEAR) * FTE_DAYS_MONTH: ,.2f}" )


    ftedays_case_count = FTE_DAYS_YEAR/case_count
    ftedays_status_change_count = FTE_DAYS_YEAR/status_change_count
    print(f"FTE Days/Adjudicated Case: {ftedays_case_count : .3f}")
    print(f"FTE Days/Status Change {ftedays_status_change_count: .3f}") 

    return (df, ftedays_case_count, ftedays_status_change_count )

In [ ]:
#exports
def example_exports_func():
    pass



In [ ]:
#exports
def imgs_save(image):
    pass



In [ ]:
#export
def get_statuses(f_path):
    file_path = f_path + 'DFC_STATUSES.csv'
    loadedstatus_df = pd.read_csv(file_path)
    loadedstatus_df = loadedstatus_df.sort_values(by = 'STATUS_CD')
    status_df = loadedstatus_df.drop(columns = ['ACTIVE_YN','UPDATE_RULE','DECISION_CODE','GROUP_ASSIGNMENT'])
    return status_df

In [ ]:

#export
def get_processed_case_data(f_path):
    
    file_path = f_path + 'cases_processed.csv'
    cases_df = pd.read_csv(file_path,parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
    cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]
    
    cases_df = cases_df[(cases_df['Is Adjudicated'] == 'Adjudicated') & (cases_df['age_bucket'] >= 80)].reset_index()

    cases_df['Origin Report'] = cases_df.apply( lambda x: ORIGIN_MAPPINGS[x['ORIGIN_DSC']] , axis=1)

    cases_df['Origin & Decision'] = cases_df.apply( lambda x: str(x['Origin Report']) + ' & ' + str(x['DECISION_DSC']), axis=1)

    cases_df['First & Second Status'] = cases_df.apply( lambda x: str(x['FIRSTSTATUSCD']) + '_' + str(x['SECOND_CASE_STATUS_CD']), axis=1)

    cases_df['DOCUMENT_COUNT'] = cases_df['DOCUMENT_COUNT'].fillna(0)

    cases_df['Age Category'] = cases_df.apply( lambda x: 'Over 80 ' if x.age_bucket >= 80 else 'Under 80', axis=1)
    cases_df['Type Origin'] = cases_df.apply( lambda x: str(x['CASE_CD']) + '_' + str(x['ORIGIN_CD']), axis=1)
    cases_df['Type & Origin Desc'] = cases_df.apply( lambda x: str(x['CASE_DSC']) + ' & ' + str(x['ORIGIN_DSC']), axis=1)
    cases_df['Case Length Over 30 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 30 else False, axis=1)
    cases_df['Case Length Over 60 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 60 else False, axis=1)


    return cases_df

    



In [ ]:

#export
def get_2018processed_case_data(f_path):
    
#     file_path = f_path + 'cases_processed.csv'
#     cases_df = pd.read_csv(file_path,parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
#     cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]
    cases_df = get_processed_case_data(f_path)
    cases2018_df = cases_df[cases_df.opened_year == 2018].reset_index()

#     cases2018_df = cases2018_df[(cases2018_df['Is Adjudicated'] == 'Adjudicated') & (cases2018_df['age_bucket'] >= 80)].reset_index()

#     cases2018_df['Origin Report'] = cases2018_df.apply( lambda x: ORIGIN_MAPPINGS[x['ORIGIN_DSC']] , axis=1)

#     cases2018_df['Origin & Decision'] = cases2018_df.apply( lambda x: str(x['Origin Report']) + ' & ' + str(x['DECISION_DSC']), axis=1)

#     cases2018_df['First & Second Status'] = cases2018_df.apply( lambda x: str(x['FIRSTSTATUSCD']) + '_' + str(x['SECOND_CASE_STATUS_CD']), axis=1)

#     cases2018_df['DOCUMENT_COUNT'] = cases2018_df['DOCUMENT_COUNT'].fillna(0)

    status_df = get_statuses(f_path)

    status_df.columns = ['SECOND_STATUS_CD', 'Second Status Desc']
    cases2018_df = pd.merge(cases2018_df, status_df, left_on='SECOND_CASE_STATUS_CD', right_on='SECOND_STATUS_CD' )

    status_df.columns = ['INITIALSTATUSCD', 'Initial Status Desc']
    cases2018_df = pd.merge(cases2018_df, status_df, left_on='FIRSTSTATUSCD', right_on='INITIALSTATUSCD' )

    cases2018_df['First & Second Status Descriptions'] = cases2018_df.apply( lambda x: str(x['Initial Status Desc']) + ' ==> ' + str(x['Second Status Desc']), axis=1)


    cases2018_df = cases2018_df.drop(columns = ['INITIALSTATUSCD', 'SECOND_STATUS_CD'])


    
    return cases2018_df



In [ ]:
#hide
df = get_fte_days_2018()
test_eq( df.shape, (12,2))

# can't use this test because it depends on an exernal file
# df = get_2018processed_case_data(file_path)
# test_ne( df.shape[0], 0)

NameError: name 'get_fte_days_2018' is not defined

In [ ]:
#hide
s = get_statuses(file_path)


In [ ]:
#columns = ['CASE_OPENED_DT','LAST_STATUS_DATE','DRIVERS_LICENSE_NO','STATUS_COUNT','Year Span','Status','Age Category']

columns = ['CASE_OPENED_DT','LAST_STATUS_DATE','DRIVERS_LICENSE_NO','STATUS_COUNT', 'Age Category', 'GENERAL_STATUS']
c = get_2018processed_case_data(file_path)
#c[columns].head()
c.head()


,level_0,index,DRIVERS_LICENSE_NO,BIRTHDATE,GENERAL_STATUS,CASE_SEQ_NO,CASE_CD,ORIGIN_CD,STATUS_CD,DECISION_CD,...,Origin & Decision,First & Second Status,Age Category,Type Origin,Type & Origin Desc,Case Length Over 30 Days,Case Length Over 60 Days,Second Status Desc,Initial Status Desc,First & Second Status Descriptions
0,3,459,0024247,1936-11-22,Closed,2,1,1,5,1,...,DMER & FIT - CURRENT CLASS,7_1.0,Over 80,1_1,ROUTINE - AGE & DMER - DRIVER MEDICAL EXAMINA...,True,True,OPEN - PENDING REVIEW,OPEN - MEDICAL ISSUED,OPEN - MEDICAL ISSUED ==> OPEN - PENDING REVIEW
1,45,3495,0183970,1934-10-12,Closed,2,6,1,5,13,...,DMER & DECEASED,7_1.0,Over 80,6_1,RUSH & DMER - DRIVER MEDICAL EXAMINATION REPORT,True,True,OPEN - PENDING REVIEW,OPEN - MEDICAL ISSUED,OPEN - MEDICAL ISSUED ==> OPEN - PENDING REVIEW
2,95,5527,0312317,1928-02-12,Closed,6,1,1,5,1,...,DMER & FIT - CURRENT CLASS,7_1.0,Over 80,1_1,ROUTINE - AGE & DMER - DRIVER MEDICAL EXAMINA...,True,True,OPEN - PENDING REVIEW,OPEN - MEDICAL ISSUED,OPEN - MEDICAL ISSUED ==> OPEN - PENDING REVIEW
3,121,5762,0325173,1933-08-17,Closed,4,4,1,5,1,...,DMER & FIT - CURRENT CLASS,7_1.0,Over 80,4_1,KNOWN CONDITION & DMER - DRIVER MEDICAL EXAMIN...,True,True,OPEN - PENDING REVIEW,OPEN - MEDICAL ISSUED,OPEN - MEDICAL ISSUED ==> OPEN - PENDING REVIEW
4,129,5850,0329459,1926-07-20,Closed,7,1,1,5,1,...,DMER & FIT - CURRENT CLASS,7_1.0,Over 80,1_1,ROUTINE - AGE & DMER - DRIVER MEDICAL EXAMINA...,True,True,OPEN - PENDING REVIEW,OPEN - MEDICAL ISSUED,OPEN - MEDICAL ISSUED ==> OPEN - PENDING REVIEW


In [ ]:
c.columns
c[columns].head()

,CASE_OPENED_DT,LAST_STATUS_DATE,DRIVERS_LICENSE_NO,STATUS_COUNT,Age Category,GENERAL_STATUS
0,2018-09-23,2020-05-27,0024247,2,Over 80,Closed
1,2018-10-07,2019-03-05,0183970,3,Over 80,Closed
2,2018-09-30,2020-05-28,0312317,2,Over 80,Closed
3,2018-07-13,2020-05-25,0325173,2,Over 80,Closed
4,2018-05-21,2020-05-20,0329459,2,Over 80,Closed


In [ ]:
#hide

from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_core.baseline.ipynb.
Converted 02_core.caseload.ipynb.
No export destination, ignored:
#exporti
data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'
cases2018_df = get_2018processed_case_data(data_file_path)

Converted 10. Cost Analysis Baseline - 2018 Over 80 Cases by Case Origin.ipynb.
Converted 20. Caseload Modelling Introduction.ipynb.
Converted 21. Caseload Modelling - Monthly Totals Cases Opened.ipynb.
Converted 22. Caseload Modelling - Monthly Totals Cases Closed.ipynb.
Converted index.ipynb.
